#### quest
 - 업무분장(전처리, 모델학습).
 - RecurrenceOfSurgery.csv 사용
 - 목표변수 범주형 : 척추전방위증 
   - 
 - 설명변수 최소 6개 : 
  - (연속형)
    체중 
    신장
        
  - (범주형)
    환자통증정도,
    스테로이드치료, 
    수술기법
    과거수술횟수
  
- 서비스 대상과 목표 설명, 변수 선택 이유

In [127]:
import pandas as pd

In [128]:
df_ROS = pd.read_csv('../datasets/RecurrenceOfSurgery.csv')
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 52 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              1894 non-null   int64  
 1   환자ID                    1894 non-null   object 
 2   Large Lymphocyte        1894 non-null   float64
 3   Location of herniation  1894 non-null   int64  
 4   ODI                     462 non-null    float64
 5   가족력                     1843 non-null   float64
 6   간질성폐질환                  1894 non-null   int64  
 7   고혈압여부                   1894 non-null   int64  
 8   과거수술횟수                  1894 non-null   int64  
 9   당뇨여부                    1894 non-null   int64  
 10  말초동맥질환여부                1894 non-null   int64  
 11  빈혈여부                    1894 non-null   int64  
 12  성별                      1894 non-null   int64  
 13  스테로이드치료                 1894 non-null   int64  
 14  신부전여부                   1894 non-null   

In [129]:
df_ROS[:2]

,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,당뇨여부,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,0,1PT,22.8,3,51.0,0.0,0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,1,2PT,44.9,4,26.0,0.0,0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [130]:
df_ROS.columns

Index(['Unnamed: 0', '환자ID', 'Large Lymphocyte', 'Location of herniation',
       'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수', '당뇨여부', '말초동맥질환여부', '빈혈여부',
       '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환', '암발병여부', '연령', '우울증여부', '입원기간',
       '입원일자', '종양진행여부', '직업', '체중', '퇴원일자', '헤모글로빈수치', '혈전합병증여부', '환자통증정도',
       '흡연여부', '통증기간(월)', '수술기법', '수술시간', '수술실패여부', '수술일자', '재발여부', '혈액형',
       '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도', 'Instability', 'MF + ES',
       'Modic change', 'PI', 'PT', 'Seg Angle(raw)', 'Vaccum disc', '골밀도',
       '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

### 특성 공학 기법

#### 1.Imputation (결측값 처리)

In [177]:
df_ROS_extract_null = df_ROS[['척추전방위증', '체중', '신장', '환자통증정도', '스테로이드치료', '수술기법', '과거수술횟수']]
df_ROS_extract_null.isnull().sum() 

척추전방위증      0
체중          0
신장          0
환자통증정도      0
스테로이드치료     0
수술기법       81
과거수술횟수      0
dtype: int64

In [178]:
df_ROS_extract.shape

(1813, 13)

In [179]:
df_ROS_extract_null[:10]

,척추전방위증,체중,신장,환자통증정도,스테로이드치료,수술기법,과거수술횟수
0,0,60.3,163,10,1,TELD,0
1,0,71.7,171,10,1,TELD,0
2,0,77.1,178,7,1,TELD,1
3,0,74.2,174,7,0,TELD,0
4,0,80.7,183,7,1,TELD,0
5,0,60.8,164,7,1,TELD,2
6,0,74.8,175,7,0,TELD,0
7,0,81.2,183,7,1,TELD,0
8,0,69.2,168,8,0,TELD,0
9,0,63.0,160,7,1,TELD,0


In [180]:
# 결측값이 있는 수술기법 목표변수로 해서 나머지는 결측치를 채우는 모델 생성 

In [181]:
# df_ROS_extract = df_ROS_extract_null.dropna()
# df_ROS_extract.isnull().sum()

척추전방위증     0
체중         0
신장         0
환자통증정도     0
스테로이드치료    0
수술기법       0
과거수술횟수     0
dtype: int64

In [182]:
df_ROS_extract['수술기법'].value_counts()

TELD    1673
IELD     140
Name: 수술기법, dtype: int64

In [183]:
df_ROS_extract.shape

(1813, 7)

In [184]:
from sklearn.preprocessing import OneHotEncoder

In [185]:
oneHotEncoder = OneHotEncoder() # 인스턴스화
oneHotEncoder.fit(df_ROS_extract[['수술기법']]) 

OneHotEncoder()

In [186]:
columns_name = oneHotEncoder.categories_

In [187]:
encoded_data = oneHotEncoder.transform(df_ROS_extract[['수술기법']]) .toarray()

In [188]:
encoded_data.shape

(1813, 2)

In [189]:
df_encoded_data = pd.DataFrame(data=encoded_data, columns=oneHotEncoder.get_feature_names_out(['수술기법']))
df_encoded_data[:2]

,수술기법_IELD,수술기법_TELD
0,0.0,1.0
1,0.0,1.0


In [190]:
df_encoded_data.index, df_encoded_data.shape

(RangeIndex(start=0, stop=1813, step=1), (1813, 2))

In [191]:
df_ROS_extract.index, df_ROS_extract.shape

(Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             1843, 1845, 1849, 1865, 1866, 1870, 1872, 1874, 1879, 1891],
            dtype='int64', length=1813),
 (1813, 7))

In [192]:
df_ROS_extract.isnull().sum()

척추전방위증     0
체중         0
신장         0
환자통증정도     0
스테로이드치료    0
수술기법       0
과거수술횟수     0
dtype: int64

In [193]:
df_ROS_extract = pd.concat([df_ROS_extract.reset_index(drop=True)
                                       , df_encoded_data.reset_index(drop=True)], axis=1)
df_ROS_extract[:2]

,척추전방위증,체중,신장,환자통증정도,스테로이드치료,수술기법,과거수술횟수,수술기법_IELD,수술기법_TELD
0,0,60.3,163,10,1,TELD,0,0.0,1.0
1,0,71.7,171,10,1,TELD,0,0.0,1.0


In [194]:
df_ROS_extract.shape

(1813, 9)

In [195]:
df_ROS_extract.columns

Index(['척추전방위증', '체중', '신장', '환자통증정도', '스테로이드치료', '수술기법', '과거수술횟수',
       '수술기법_IELD', '수술기법_TELD'],
      dtype='object')

In [198]:
target = df_ROS_extract['척추전방위증']

In [199]:
features = df_ROS_extract.drop(columns=['척추전방위증', '수술기법',])

In [200]:
features.columns

Index(['체중', '신장', '환자통증정도', '스테로이드치료', '과거수술횟수', '수술기법_IELD', '수술기법_TELD'], dtype='object')

#### MinMaxScaler

In [201]:
from sklearn.preprocessing import MinMaxScaler

In [202]:
minMaxScaler = MinMaxScaler() # 인스턴스화
features= minMaxScaler.fit_transform(features)
features.shape

(1813, 7)

#### 결측치 모델 학습 

In [203]:
from sklearn.linear_model import LogisticRegression
Imputation_model = LogisticRegression()
Imputation_model.fit(features, target)

LogisticRegression()

In [204]:
Imputation_model.coef_, Imputation_model.intercept_

(array([[ 0.25462964, -0.65154355,  0.19461465,  0.0336464 ,  1.00137348,
         -0.51514264,  0.51503592]]),
 array([-4.0260358]))

In [205]:
features[:2]

array([[0.25024728, 0.359375  , 1.        , 1.        , 0.        ,
        0.        , 1.        ],
       [0.36300692, 0.484375  , 1.        , 1.        , 0.        ,
        0.        , 1.        ]])

In [206]:
Imputation_model.predict(features[:1])

array([0], dtype=int64)

In [207]:
# def 함수 (row) :
#     -ow 에 비교했을때 null값이 있다면 model 함수를 적용해서 null값을 뱉어라
#     -아니면 그대로 뱉어라
    
#     .apply(합수)


# def preprocess_and_train(features):
#     result = Imputation_model.predict(features)
#     return result



In [ ]:
# 1. target : '수술시간', feature: '성별'
target = df_ROS_extract[['수술기법']]
feature = df_ROS_extract[['환자통증정도']]
target.shape, feature.shape

In [110]:
# apply()
def convert_notnull(row) :
    if row == None :
        result = Imputation_model.predict(row)
        return result
    else :
        result1 = df_ROS_extract_null['수술기법']
        return result1

In [112]:
df_ROS_extract_null['수술기법'].apply(convert_notnull)

ValueError: Expected 2D array, got 1D array instead:
array=[0 0 0 ... 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
### Under Sampling